In [ ]:
from datetime import datetime, timedelta
import os 
import pickle

import geopandas as gpd
import matplotlib.pyplot as plt
from tqdm import tqdm

from gee import DataConfig
import get_training_data

%load_ext autoreload
%autoreload 2

WORK_DIR = '../data/sampling_locations/'

In [ ]:
timestamp = '2025-08-28T15:10'
gdf = gpd.read_file(os.path.join(WORK_DIR, f"collected_locations{timestamp}.geojson"))
gdf.head()
    

In [ ]:
gdf.clear_threshold.value_counts()

In [ ]:
gdf[['split', 'label']].value_counts()

In [ ]:
clear_threshold = 0.75 # minimum clear pixels for a tile to be included in the dataset

config = DataConfig(
    clear_threshold = clear_threshold,  
    tilesize = 48,
    collection = 'S2L1C'
)
resolution = 10
outdir = f"../data/training_patches{timestamp}"

training_data = get_training_data.TrainingData(config, resolution, outdir)
    
#tiles = training_data.fetch_tiles_for_points(gdf[gdf.clear_threshold == clear_threshold])
tiles = training_data.fetch_tiles_for_points(gdf[~gdf.source_file.str.contains('random') & (gdf.clear_threshold == clear_threshold)])


In [ ]:
# 25k random negatives take a few hours - so run separately
tiles = training_data.fetch_tiles_for_points(gdf[gdf.source_file.str.contains('random') & (gdf.clear_threshold == clear_threshold)])


In [ ]:
# Cloudy negatives 
clear_threshold = 0.1
config.clear_threshold = clear_threshold
training_data = get_training_data.TrainingData(config, resolution, outdir)
    
tiles = training_data.fetch_tiles_for_points(gdf[gdf.clear_threshold == clear_threshold])